# AIAB Braitenberg Report

<img src="http://users.sussex.ac.uk/~christ/crs/kr-ist/copied-pics/braitenberg-vehicles-fear-v-aggression.png" width="20%" style="display: block;margin-left: auto;margin-right: auto;" >

This colab worksheet contains the complete code for the simulation of the Braitenberg robots for the AIAB Final Project


## Installing libraries
Firstly we need to install the library for interfacing with the robots, and import all the required libraries


In [ ]:
#!pip install mpremote
import numpy as np
import matplotlib.pyplot as plt
#from mpremote import pyboard
from copy import deepcopy
#import serial, glob, sys
import random

## Setting up the braitenberg and environment

The braitenberg can have a genotype encoded to represent the different parts, and is always given starting positions


In [ ]:
class Braitenberg:
  """
  This class holds the genotype and position of the braigtenberg vihicle
  """
  def __init__(self,initial_pos,initial_bearing,geno):
    #@ param initial_pos is the starting pos
    #@ param initial_bearing is the starting direction of the board (degrees) e.g 45
    #@ param geno is the genotype to encode hardware e.g [w_ll w_lr w_rl w_rr bl br] encoded as 1 or 0
    self.geno=geno
    #assert (type(self.geno)==type([]) or type(self.geno)==type(np.array([]))) and len(self.geno)==6,"Genotype must be an array of size 6"
    self.initial_bearing = initial_bearing
    self.pos=initial_pos
  def get_geno(self):
    #@ returns the genotype options
    return self.geno

class environment:
  """
  allows running of a trial in a set up environment
  """
  def __init__(self, sig=0.1):
    # Initial setup
    self.dt=0.05; # How fast time moves
    self.R =0.05; #radius, size of the vehicle
    self.b = 45; #(degrees) sensor angle
    self.sig = sig

  def run(self,T,agent,motor_gain=0.5,show=True,S=(0,0)):
    #@ param T is the time it will run for
    #@ param agent is the braitenberg vehicle object used
    #@ param show is whether it will show the plot
    #@ returns 2D array of positions the agent has traveleld through [[x],[y]]
    #conver geno params
    w_ll,w_lr,w_rl,w_rr,bl,br = agent.get_geno()
    sl_pos = np.zeros((2,1));
    sr_pos = np.zeros((2,1));
    S=np.array(S)
    rho=np.zeros((2, 1));
    sensor_gain = 0.5; # How sensitive the vehicle is to light. Higher = more
    motor_gain =motor_gain; # How much vehicle responds to light. Higher = faster
    vl=0;vr=0; # Speed of the left and right wheels
    #convert to radians

    initial_bearing = agent.initial_bearing/360*2*np.pi;
    b=self.b/360*2*np.pi;

    pos = np.zeros((2,int(T/self.dt)));
    bearing = np.zeros((1,int(T/self.dt)));
    pos[:,0] = agent.pos;
    bearing[:,0] = initial_bearing;
    lightIntensity=np.zeros((2,int(T/self.dt)));

    for i in range(1, int(T/self.dt)):
        vc = (vl+vr)/2; # Calculate the average speed of the wheels
        va = (vr-vl)/(2*self.R); # Calculate how much the vehicle should turn

        # Update the position based on curr velocity and direction
        # New pos = Previous position + horizontal distance traveled
        pos[0,i] = pos[0,i-1]+ self.dt*vc*np.cos(bearing[0,i-1]);
        # New pos = prev pos + vertical distance traveled
        pos[1,i] = pos[1,i-1]+ self.dt*vc*np.sin(bearing[0,i-1]);
        # New angle = prev angle + change in angle
        bearing[0,i] = np.mod(bearing[0,i-1] + self.dt*va,2*np.pi);

        # Calculate left sensor position
        sl_pos[0] = pos[0,i] + self.R*np.cos(bearing[0,i]+b);
        sl_pos[1] = pos[1,i] + self.R*np.sin(bearing[0,i]+b);

        # Calculate right sensor position
        sr_pos[0] = pos[0,i] + self.R*np.cos(bearing[0,i]-b);
        sr_pos[1] = pos[1,i] + self.R*np.sin(bearing[0,i]-b);

        # Calculate (square) distance to element
        #this code assumes light is always at S
        dl = np.sqrt((sl_pos[0]-S[0])**2+(sl_pos[1]-S[1])**2);
        dr = np.sqrt((sr_pos[0]-S[0])**2+(sr_pos[1]-S[1])**2);

        #  Calculate local intensity
        il = sensor_gain/dl;
        ir = sensor_gain/dr;
        lightIntensity[0,i] = il
        lightIntensity[1,i] = ir

        #weights times inputs plus bias
        # The brain, figuring out if it should go straight or turn
        lm = il*w_ll + ir*w_rl + bl;
        rm = il*w_lr + ir*w_rr + br;
        # add a some gaussian noise to ech motor.
        # The sig variable controls how much noise is added
        # Adding a little bit of randomness (maybe overturn left or right)
        lm = lm + np.random.normal(0, self.sig)
        rm = rm + np.random.normal(0, self.sig)

        #  Scale by motor gains
        # Setting how fast each wheel should go
        vl =motor_gain*lm;
        vr =motor_gain*rm;
    if show:
      self.show(T,pos,sl_pos,sr_pos,bearing,b)
    return pos,lightIntensity
  def show(self,T,pos,sl_pos,sr_pos,bearing,b):
    plt.plot(pos[0,:],pos[1,:])
    #final postion
    x=pos[0,int(T/self.dt)-1];
    y= pos[1,int(T/self.dt)-1];
    f_bearing = bearing[0,int(T/self.dt)-1];
    # Calculate left sensor position
    sl_pos[0] = x + self.R*np.cos(f_bearing+b);
    sl_pos[1] = y + self.R*np.sin(f_bearing+b);
    # Calculate left sensor position
    sr_pos[0] = x + self.R*np.cos(f_bearing-b);
    sr_pos[1] = y + self.R*np.sin(f_bearing-b);

    plt.plot(0,0,marker='.',markersize=30,color='yellow');
    plt.plot(0,0,marker='o',markersize=10,color='black');

    # Plot  sensors
    plt.plot(sl_pos[0],sl_pos[1],marker='.',markersize=10,color='red');
    plt.plot(sr_pos[0],sr_pos[1],marker='.',markersize=10,color='red');

    # Plot body
    plt.plot( x, y,marker='.',markersize=10,color='blue');
    plt.plot(x,y,marker='o',markersize=10,color='black');
    #Plot trajectory
    plt.pause(0.05)

    return pos

## Generate a genotype


In [ ]:
def generate_genotype():
  # The connection weights (wires) are random floating point numbers in the interval [0, 5)
  return np.random.rand(6)*5

genotype=generate_genotype()
print("This is what a genotype looks like",genotype)

# set the starting position and the bearing
starting_position = (3,3)
starting_bearing = 10 #bearing in degrees
runtime = 5 #run to 5 seconds

## Initialization for evolving a solution

In [ ]:
# Initialization
generations = 1000
num_genes = len(genotype)
num_individuals = 50
p_crossover = 0.5
k = 3 # define local neighbourhood
lightsource = (0,0)


def euclid_dist(p1, p2):
  """
  Calculate the euclidean distance between pairs of points
  :return: distance of points
  """
  return np.sqrt(np.sum(np.square(p1 - p2), axis=-1))

#1.1
def fitness(positions,intensities,lightsource=(0,0)):
  # Goal: Sum up all of the intensities

  total_intensity = 0

  # Left intensities
  for i in range (0, len(intensities[0])):
    total_intensity += intensities[0][i]
  # Right intensities
  for i in range (0, len(intensities[1])):
    total_intensity += intensities[1][i]

  # Calculate fitness
  fitness = total_intensity

  # Return fitness
  return fitness
  pass

#1.2
def mutate(genotype, mean, standard_d):
  mutation_r = 0.3
  copy_geno = np.copy(genotype)

  # By some mutation rate, mutate the genes
  rand = np.random.uniform(0,1)
  if (rand < mutation_r):
    # Add gaussian noise according to mean and standard deviation input
    copy_geno += np.random.normal(mean, standard_d, size=genotype.shape)

  for i in range(num_genes):
    # Ensure that the gene values are limited to not be too high or small
    # Can't be 5 or higher or less than 0 (by genotype creation)
    if copy_geno[i] > 4.99:
      copy_geno[i] = 4.99
    if copy_geno[i] < 0:
      copy_geno[i] = 0

  #return mutated genotype
  return copy_geno
  pass

################################################################################

## Hill Climber with 1 individual

In [ ]:
################################################################################

# Define testing parameters
means = [1, 1, 1]
std_devs = [0.1, 0.5, 0.9]

generations = 1000

# Graph variables
x_array = []

# x = Generations
for i in range(0, generations):
  if i % 100 == 0:
    x_array.append(i)

# y = fitness by standard deviation
y_01 = []
y_05 = []
y_09 = []

# 1.3 Implement a hill climber that evolves the genotype
num_individuals = 1

# set the starting position and the bearing
starting_position = (3,3)
starting_bearing = 10 #bearing in degrees
runtime = 5 #run to 5 seconds

# Run it some amount of times (subject to change)
for trial in range(0, 1):

  # Start each run with the same genotypes
  beg_genotypes = np.zeros((num_individuals, num_genes))
  for i in range(num_individuals):
    beg_genotypes[i] = generate_genotype()

  # Run for each testing parameter (each standard deviation)
  for run in range(0,3):

    genotypes = np.copy(beg_genotypes)

    # Number of generations
    for g in range(0, generations):

      # Copy the genotype to edit
      copy_geno = np.copy(genotypes)

      # Set up all the agents
      agents = []
      for i in range(num_individuals):
        agent = Braitenberg(starting_position, starting_bearing, genotypes[i])
        agents.append(agent)

      # Create the environment that the agent will run in
      env=environment()

      # Calculate original fitness
      og_fit = np.zeros(num_individuals)
      avg_fitness = 0
      for x in range(num_individuals):
        # Get the average fitness over some amount of runs (subject to change)
        for r in range(0, 1):
          trajectory,intensities = env.run(runtime, agents[x],show=False)
          avg_fitness += fitness(trajectory, intensities, lightsource)
        avg_fitness = avg_fitness / 1
        og_fit[x] = avg_fitness

      # For however many individuals there are
      for i in range(num_individuals):
        # Mutate the genotype
        copy_geno[i] = mutate(genotypes[i], means[run], std_devs[run])

      # Set up all the new agents
      agents = []
      for i in range(num_individuals):
        agent = Braitenberg(starting_position, starting_bearing, copy_geno[i])
        agents.append(agent)

      # Calculate the new fitness
      new_fit = np.zeros(num_individuals)
      avg_fitness = 0
      for x in range(num_individuals):
        # Get the average fitness over some amount of runs (subject to change)
        for r in range(0, 1):
          trajectory,intensities = env.run(runtime, agents[x],show=False)
          avg_fitness += fitness(trajectory, intensities, lightsource)
        avg_fitness = avg_fitness / 1
        new_fit[x] = avg_fitness

      # If the new fitness is better than the old, replace that line
      # Add the fitness to the y's for the graph
      successful_individuals = 0
      for x in range(num_individuals):
        if new_fit[x] > og_fit[x]:
          genotypes[x] = copy_geno[x]
          successful_individuals += 1
          if (g % 100 == 0):
            if (run == 0):
              y_01.append(new_fit[x])
            if (run == 1):
              y_05.append(new_fit[x])
            if (run == 2):
              y_09.append(new_fit[x])
        if (new_fit[x] <= og_fit[x]):
          if (g % 100 == 0):
            if (run == 0):
              y_01.append(og_fit[x])
            if (run == 1):
              y_05.append(og_fit[x])
            if (run == 2):
              y_09.append(og_fit[x])

    # Find the best genotype
    best_idx = 0
    for x in range(num_individuals):
      if og_fit[x] > og_fit[best_idx]:
        best_idx = x

    # Run and plot the best genotype
    agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
    trajectory,intensities = env.run(runtime, agent,show=False)
    print("Trial ", trial, " Stdev ", std_devs[run])
    print("Genotype of the best individual: ", genotypes[best_idx])

    iterations = 1000
    total_fitness = 0

    # Run the best genotype over a number of trials to get the average fitness
    for i in range(iterations):
      agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
      trajectory,intensities = env.run(runtime, agent,show=False)
      total_fitness += fitness(trajectory, intensities, lightsource)

    avg_fitness1 = total_fitness / iterations
    print("Average fitness: ", avg_fitness1, "\n")

  # Plotting all lines with specifying labels
  plt.plot(x_array, y_01, label='Stdev 0.1')
  plt.plot(x_array, y_05, label='Stdev 0.5')
  plt.plot(x_array, y_09, label='Stdev 0.9')
  # Adding legend, x and y labels, and titles for the lines
  plt.legend()
  plt.xlabel('Generations')
  plt.ylabel('Fitness')
  plt.title('Fitness over time with Various Mutation Standard Deviations')
  # Displaying the plot
  plt.show()

## Full Microbial GA

In [ ]:
################################################################################
################################################################################

# Define the testing parameters
means = [1, 1, 1]
std_devs = [0.1, 0.5, 0.9]

generations = 1000

# Initialization
k = 3 # define local neighbourhood
num_individuals = 50
p_crossover = 0.5

# Run it for some number of trials (subject to change)
for trial in range(0, 1):

  # Set the genotypes to be constant for all 3 runs
  beg_genotypes = np.zeros((num_individuals, num_genes))
  for i in range(num_individuals):
    beg_genotypes[i] = generate_genotype()

  # Run it for each testing parameter (each standard deviation)
  for run in range(0,3):

    # 1.4 Implement full Microbial GA
    genotypes = np.copy(beg_genotypes)

    # Loop through the generations
    for g in range(generations):

      # 3) Pick one individual at random, i.e. Genotype 𝐺1 at position 𝑥1
      G1 = np.random.randint(0, num_individuals - 1)

      # 4) Pick a second individual  𝐺2  in the local neighbourhood of the first, i.e.,
      #    pick a competitor from the local neighbourhood in the range  𝑥1+1  to  𝑥1+𝑘
      #    Find winner and loser
      G2 = np.random.randint(G1+1, G1 + k)
      if G2 >= num_individuals:
        G2 = G2 - num_individuals

      # Set up all the agents
      agents = []
      for i in range(num_individuals):
        agent = Braitenberg(starting_position, starting_bearing, genotypes[i])
        agents.append(agent)

      # Create the environment that the agent will run in
      env=environment()

      # Calculate random individual 1 fitness
      trajectory,intensities = env.run(runtime, agents[G1],show=False)
      G1_fit = fitness(trajectory, intensities, lightsource)
      # Calculate random individual 2 fitness
      trajectory,intensities = env.run(runtime, agents[G2],show=False)
      G2_fit = fitness(trajectory, intensities, lightsource)

      # Pick a winner and a loser
      # 5) Copy each gene of the winner W to the L with crossover probability
      if (G1_fit > G2_fit):
        for idx in range(num_genes):
          # Pick a random number
          rand = np.random.uniform(0,1)
          if (rand < p_crossover):
            genotypes[G2][idx] = genotypes[G1][idx]
        l = G2
      else:
        for idx in range(num_genes):
          # Pick a random number
          rand = np.random.uniform(0,1)
          if (rand < p_crossover):
            genotypes[G1][idx] = genotypes[G2][idx]
        l = G1

      # 6) Add a mutation to L, insert it back to the population.
      genotypes[l] = mutate(genotypes[l], means[run], std_devs[run])

    # 7) Until success or give up, goto 3

    ################################################################################
    # END ALGORITHM

    # Find the best genotype
    best_idx = 0

    # Set up all the agents with final genotypes
    agents = []
    for i in range(num_individuals):
      agent = Braitenberg(starting_position, starting_bearing, genotypes[i])
      agents.append(agent)

    # Calculate fitness of all of the agents
    og_fit = np.zeros(num_individuals)

    for x in range(num_individuals):

      # Get the average fitness over 10 trials
      # We do this because some iterations have a way better fitness than others
      iterations = 10
      total_fitness = 0

      # Loop through the iterations
      for i in range(iterations):
        trajectory,intensities = env.run(runtime, agents[x],show=False)
        total_fitness += fitness(trajectory, intensities, lightsource)

      # Take an average of the fitness
      avg_fitness = total_fitness / iterations

      # Record the average fitness for each genotype
      og_fit[x] = avg_fitness

    # Find the genotype index with the best fitness
    for x in range(num_individuals):
      if og_fit[x] > og_fit[best_idx]:
        best_idx = x

    # Run and plot the best genotype
    agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
    trajectory,intensities = env.run(runtime, agent,show=False)
    print("Trial ", trial, " Stdev: ", std_devs[run])
    print("Genotype of the best individual: ", genotypes[best_idx])

    iterations = 1000
    total_fitness = 0

    # Run the best genotype over a number of iterations to get the average fitness
    for i in range(iterations):
      agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
      trajectory,intensities = env.run(runtime, agent,show=False)
      total_fitness += fitness(trajectory, intensities, lightsource)

    avg_fitness1 = total_fitness / iterations
    print("Average fitness: ", avg_fitness1, "\n")

### A commentary on the code above
Although it looks like the genotypes produced by this algorithm are doing really well, upon testing it on the robot, they are only doing so for the left sensor. This is likely because we are training it on one set bearing with a constant starting position and lightsource. As a result, the algorithm might just be learning the path. This is overfitting.

We can fix this by adding a random bearing to our algorithm and a random starting position on each iteration, making sure that the genotypes we compare in tournament selection are using the same environment. This should fix this overfitting (future plans)

## Testing p crossover with Full Microbial

In [ ]:
################################################################################
################################################################################

# We've learned that stdev of 0.9 works best (barely, but a little)
# So keep stdev at 0.9, try changing crossover now

# Define testing parameters
p_crossovers = [0.1, 0.5, 0.9]

generations = 1000

# Initialization
k = 3 # define local neighbourhood
num_individuals = 50

# Run for some number of trials (subject to change)
for trial in range(0, 1):

  # Set the genotypes to be constant for all 3 runs
  beg_genotypes = np.zeros((num_individuals, num_genes))
  for i in range(num_individuals):
    beg_genotypes[i] = generate_genotype()

  # Run for each testing parameter (each crossover probability)
  for run in range(0,3):

    # 1.4 Implement full Microbial GA
    genotypes = np.copy(beg_genotypes)

    # Loop through the generations
    for g in range(generations):

      # 3) Pick one individual at random, i.e. Genotype 𝐺1 at position 𝑥1
      G1 = np.random.randint(0, num_individuals - 1)

      # 4) Pick a second individual  𝐺2  in the local neighbourhood of the first, i.e.,
      #    pick a competitor from the local neighbourhood in the range  𝑥1+1  to  𝑥1+𝑘
      #    Find winner and loser
      G2 = np.random.randint(G1+1, G1 + k)
      if G2 >= num_individuals:
        G2 = G2 - num_individuals

      # Set up all the agents
      agents = []
      for i in range(num_individuals):
        agent = Braitenberg(starting_position, starting_bearing, genotypes[i])
        agents.append(agent)

      # Create the environment that the agent will run in
      env=environment()

      # Calculate random individual 1 fitness
      trajectory,intensities = env.run(runtime, agents[G1],show=False)
      G1_fit = fitness(trajectory, intensities, lightsource)
      # Calculate random individual 2 fitness
      trajectory,intensities = env.run(runtime, agents[G2],show=False)
      G2_fit = fitness(trajectory, intensities, lightsource)

      # Pick a winner and a loser
      # 5) Copy each gene of the winner W to the L with crossover probability
      if (G1_fit > G2_fit):
        for idx in range(num_genes):
          # Pick a random number
          rand = np.random.uniform(0,1)
          if (rand < p_crossovers[run]):
            genotypes[G2][idx] = genotypes[G1][idx]
        l = G2
      else:
        for idx in range(num_genes):
          # Pick a random number
          rand = np.random.uniform(0,1)
          if (rand < p_crossovers[run]):
            genotypes[G1][idx] = genotypes[G2][idx]
        l = G1

      # 6) Add a mutation to L, insert it back to the population.
      genotypes[l] = mutate(genotypes[l], 1, 0.9)

    # 7) Until success or give up, goto 3

    ################################################################################
    # END ALGORITHM

    # Find the best genotype
    best_idx = 0

    # Set up all the agents with final genotypes
    agents = []
    for i in range(num_individuals):
      agent = Braitenberg(starting_position, starting_bearing, genotypes[i])
      agents.append(agent)

    # Calculate fitness of all of the agents
    og_fit = np.zeros(num_individuals)

    for x in range(num_individuals):

      # Get the average fitness over 10 trials
      # We do this because some iterations have a way better fitness than others
      iterations = 10
      total_fitness = 0

      # Loop through the iterations
      for i in range(iterations):
        trajectory,intensities = env.run(runtime, agents[x],show=False)
        total_fitness += fitness(trajectory, intensities, lightsource)

      # Take an average of the fitness
      avg_fitness = total_fitness / iterations

      # Record the average fitness for each genotype
      og_fit[x] = avg_fitness

    # Find the genotype index with the best fitness
    for x in range(num_individuals):
      if og_fit[x] > og_fit[best_idx]:
        best_idx = x

    # Run and plot the best genotype
    agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
    trajectory,intensities = env.run(runtime, agent,show=False)
    print("Trial ", trial, " P-crossover: ", p_crossovers[run])
    print("Genotype of the best individual: ", genotypes[best_idx])

    iterations = 1000
    total_fitness = 0

    # Run best genotype for some number of iterations to get average fitness
    for i in range(iterations):
      agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
      trajectory,intensities = env.run(runtime, agent,show=False)
      total_fitness += fitness(trajectory, intensities, lightsource)

    avg_fitness1 = total_fitness / iterations
    print("Average fitness: ", avg_fitness1, "\n")

## To test genotypes

In [ ]:
# TO TEST GENOTYPES

# Change this
test_genotype = [0, 0, 0, 0, 1, 1]

# Leave this
starting_bearing = 10
starting_position = (0,0)
iterations = 1000
total_fitness = 0
env=environment() #create the environment that the agent will run in

# Get the average fitness over a number of iterations
for i in range(iterations):
  agent = Braitenberg(starting_position, starting_bearing, test_genotype)
  trajectory,intensities = env.run(runtime, agent,show=False)
  total_fitness += fitness(trajectory, intensities, lightsource)

# Print average fitness
avg_fitness = total_fitness / iterations
print(avg_fitness)


## Graphs

In [ ]:
# Hillclimber mutation noise

y1_dots = [26.72392644,26.84111002,20.19094845,25.48894085,26.67082247,26.67478738,17.10068673,25.82542233,30.79899128,17.99034429,21.04870486,20.06460507,21.89418381,20.87848844,32.16361849,27.61684211,18.45259021,18.14569067,25.15349917,30.77434631,21.86202724,21.08482003,10.74041695,29.74101166,25.21749787,22.06049728,18.31222963,21.04417939,28.43691204,25.58419515,20.38455968,21.21845871,20.36296346,21.11144126,25.38205121,27.25568963,35.7689583,26.9317957,21.67552318,25.32349024,30.61261944,26.32318818,19.21551335,25.62500478,22.06377929,20.20976564,19.75532395,43.24364544,21.15991811,21.17180099,27.75156685,18.15115909,30.24662422,25.49578194,14.64516483,18.10612894,21.04189207,22.0891088,25.34146582,13.79017523,33.98220936,20.1992366,31.48058661,31.88826663,21.07970647,20.22131901,21.59258896,21.00717987,24.86104534,22.0125293,21.5846822,35.85883421,20.36040033,21.06986999,25.50394048,21.48170273,26.56526661,17.55729483,10.73229939,12.13062887,27.6754476,25.62585706,23.56890382,25.75420117,26.97176654,34.92782021,21.20182483,19.49049788,21.51417518,21.06914831,32.02721324,20.80945925,21.65489106,21.47098986,18.41410297,21.83861444,21.56807899,25.32740776,21.23270704,25.29903582,20.9587493,17.83136311,26.92617796,21.39218111,20.62943389,25.70320066,18.42180781,27.9229115,21.60756922,20.27292594,20.21379647,20.93133344,29.70475761,15.00536007,21.4857992,10.73298763,28.02764727,32.65722446,13.37143518,20.61080817,32.70982307,12.11640442,31.46703501,18.50152546,25.50689878]
y2_dots = [27.88713266,26.96298916,30.49901627,29.65505536,31.15738066,32.02237898,27.51639063,31.70850098,32.23352538,31.9336363,19.67576061,23.90214101,27.3896721,31.95802475,32.48775513,31.8155108,33.90226807,30.30774227,27.37557255,31.59451959,27.43188556,21.03401984,30.21981992,28.26586854,32.398902,29.69880557,22.01143216,26.14478503,31.37000141,27.52331534,35.63603798,30.83099972,29.62297909,31.25993735,32.24889723,29.0588721,30.29513463,26.9575969,26.25256876,31.52589473,25.25305681,30.04973877,31.21812521,33.67638022,30.91987596,28.98705378,20.23205517,35.1115468,28.13441694,27.04625121,36.71859414,31.42619653,27.51851543,31.64199733,31.85073713,34.24921335,32.0694422,32.01894429,17.31336213,31.98211188,31.77805466,25.70317059,31.52320389,26.26172552,31.41346271,27.18681428,31.64209481,31.81055191,20.30290064,31.05092952,28.87393973,31.17211057,26.89279254,31.75961934,36.85925754,20.18233566,31.19028626,32.03474949,31.53790934,26.37852532,32.36392012,28.436358,29.62451743,34.54794471,31.98879558,31.51141613,31.19681792,26.31334811,30.06337483,21.3525269,32.16490682,20.20134271,32.02406697,30.90559358,30.54549374,23.22997192,29.2684881,31.7742394,26.70591586,31.97225009,18.05877762,28.37960632,31.52797412,24.92825505,32.41379943,31.20658504,21.51497119,27.77832874,19.15226474,18.15930649,31.93277268,31.95931933,34.57203439,34.35466315,32.09471087,31.37237013,29.67099307,27.99176546,31.96258924,31.42600562,29.81086156,28.86905194,31.3248659,30.03187674,29.29042864]
y3_dots = [29.53032238,37.337382,31.84891009,25.51120299,37.35891563,29.97751946,33.74121245,27.12233107,26.92021703,26.72409535,32.18833356,30.29233925,26.98859219,31.87512329,32.20645807,39.54426066,31.27700401,26.23173598,33.20615549,27.1937472,27.20359638,34.89280208,29.87801434,29.26738107,30.59202598,32.57860656,22.19423013,30.71553258,26.59984707,22.31526907,26.60591092,29.95202218,22.91589992,19.60829983,27.22048048,30.62884156,26.71895799,23.70050679,29.11892835,24.49949759,28.89363022,27.19696389,27.47436888,35.3005389,34.64476928,22.90807964,32.99389399,27.06102967,30.71820589,22.06792215,27.58650108,34.84885689,27.08834619,35.56253435,31.91666249,29.47313934,28.00834922,29.50300196,29.87019089,26.36639921,31.65237535,24.20722711,26.95738025,32.00144545,31.77449829,22.65143394,26.4198313,26.81812622,29.71539221,30.80558312,25.30189401,33.63909002,30.04567437,27.54419163,28.15036967,26.62087496,32.2487595,31.87185366,27.62173262,31.73830219,34.37956099,31.71592367,25.0581476,36.61513972,33.33120206,32.28055773,24.12616108,35.64190708,25.65295635,25.56396431,32.14366363,27.86515164,31.74982597,37.24005828,30.7715899,25.36169576,31.41704312,32.45863466,36.37073733,31.96051233,27.51991201,22.5326088,27.47382469,26.6232824,30.49626342,29.62247584,29.18253049,27.35457416,32.26430843,32.64263269,21.05995693,29.70475761,25.66836808,31.15105089,31.96944915,29.73804616,30.94802268,27.48531143,31.36363176,27.59220681,27.28657139,30.90643578,27.59823341,26.12084375,35.8072459]

x_dots = []

for i in range(1, 126):
  x_dots.append(i)

# Plot all lines
plt.plot(x_dots, y1_dots, label='Stdev 0.1')
plt.plot(x_dots, y2_dots, label='Stdev 0.5')
plt.plot(x_dots, y3_dots, label='Stdev 0.9')
# Add legend, x and y labels, and titles for the lines
plt.legend()
plt.xlabel('Trials')
plt.ylabel('Average Fitness')
plt.title('Average Fitness per Trial in Hillclimber Algorithm')
# Display the plot
plt.show()

In [ ]:
# Full microbial mutation noise

y1_dots = [71.84276368,68.70710781,43.63063243,43.82996267,41.00161689,78.29512518,41.03581828,55.78057513,81.40880483,43.7562623,74.56198021,66.53066703,64.42553753,38.99838632,37.30233417,74.08002241,61.40872432,36.4649158,43.7079537,73.29986078,78.09728667,40.11274092,68.88957891,56.90333452,68.8924143,37.64430701,45.92294062,42.24038565,65.97787561,61.31469764,42.43673241,62.29374411,41.08571195,66.59666059,59.59579938,40.81423733,47.83451057,61.8645466,65.54630636,74.30361808,72.90830299,37.99250411,67.23116253,63.81240252,56.41460473,71.86509174,62.01788609,63.5523523,51.48927184,70.93428407,54.08092159,50.77044366,58.67974896,38.5194833,54.39230806,66.81949406,68.25077967,58.25066443,40.92952207,67.02672534,64.70463472,58.82369591,69.70733972,58.79502377,54.59722477,61.8216864,58.56527187,64.68396252,64.07596919,57.03577619,65.92024557,67.83742843,44.09985031,64.60015166,44.82808861,61.61079347,67.08417204,53.88427725,74.72540014,50.8790912,78.53897916,79.41408737,50.36340202,57.37269878,52.79251011,54.49095796,60.98366479,58.96363294,65.00180579,48.05882788,39.89561417,65.16981086,57.64643404,60.10932499,50.47054352,70.03492546,50.5806999,63.4290492,58.75348592,45.86278133,59.68332132,58.48004635,38.72013686,58.99906026,69.68775708,39.95841429,48.44777533,78.55847313,55.12943292,44.25691856,57.40615887,39.97097082,65.55708957,40.92490623,55.47447042,44.92511865,56.44566391,76.3146261,56.92530404,50.18112021,48.56578761,45.19684545,78.93983263,65.41311532,57.23570451]
y2_dots = [49.94911682,58.76454556,58.7579925,64.1311847,65.19707167,82.32807685,49.42992116,79.33187965,46.21185495,63.41809179,58.9546693,38.19141168,42.1477593,61.45473944,68.34394733,57.90601168,78.35002118,65.61276923,45.49150117,49.79813514,60.82805209,70.73207376,39.13163969,57.00828021,43.67863843,66.91418931,56.22574292,57.50639119,57.15542698,76.07477512,77.13511534,40.20752093,69.73058224,47.05468216,58.46936454,69.46683667,67.5567385,76.60197366,47.72439003,70.6067352,70.75916136,82.53777979,65.88586619,68.5871361,36.72815099,53.70173394,58.77007009,63.29025886,74.21682022,51.68550185,59.56463846,58.70824185,52.78073396,70.18194048,57.41173918,65.15409414,47.81734866,58.81700673,58.68059612,68.51412939,61.85099306,42.00000522,61.38223462,44.66894898,70.8456719,81.01728642,50.64330473,38.58074642,68.70051001,58.6291881,52.93631917,71.18656973,62.85005077,38.7279273,65.2331183,68.46080835,79.13704422,65.95019736,56.63199095,59.84689564,82.38012908,40.75820239,68.43887787,59.53948441,55.86213867,48.88048389,41.54085702,44.57493556,47.0037385,64.3722723,49.0366071,71.96428933,61.30301243,61.31267342,43.46772553,57.59524078,50.67208628,60.40576559,53.13380035,57.44274767,58.68729179,72.1664629,36.69430916,61.35792534,64.12220962,58.07111492,46.08521466,72.36529273,66.4727637,62.64125153,73.02182581,78.99550459,59.15512841,50.17308232,66.45295124,59.56020262,42.41987128,55.0980864,38.45938094,78.93913673,45.54033216,66.68021753,51.20537627,54.2476446,54.72777722]
y3_dots = [53.69902501,71.06205617,54.6246019,50.96949039,37.78494339,65.99006271,59.29380037,64.01406791,69.65561831,78.23301584,59.21018626,49.67697126,48.43818378,67.06782178,69.3537659,67.75514771,42.67418724,80.12407198,63.51349456,68.51353573,80.77601882,55.67599762,53.00321137,65.03074524,56.65666579,56.37322046,74.18878562,67.5266582,66.8061247,74.3321075,64.24236873,42.20946409,70.14237532,60.12159656,65.10327981,79.63618302,59.91131957,60.24838048,52.24482578,48.83561749,57.26201148,36.20635975,56.85245172,57.18567479,54.91298719,50.50199101,61.72374863,42.48375703,58.39911568,41.33450622,72.92551361,47.52987764,54.29887411,47.3486837,75.82588084,75.33077253,58.76371705,59.6337764,62.05720159,64.20533491,45.92593388,59.34074345,77.81460887,54.66965778,60.60079707,45.86113467,58.63815996,49.28553992,71.63103004,62.2082158,47.0704549,67.89857848,40.1088925,50.74142795,62.75691786,56.6935553,79.07567666,49.52134098,62.34860968,48.93449873,46.2997796,58.3697176,49.45552592,66.98520773,59.32350695,57.16135836,80.3056426,63.40713049,63.13275946,62.44149733,53.54349638,58.32834573,53.62426943,45.22125066,58.09879667,74.46550742,81.68244941,63.71629939,62.30273174,61.66937621,48.126443,81.43912777,65.435749,76.09096361,59.3250858,65.9721525,51.53643244,68.38377493,35.75514399,64.54178525,60.0949956,52.59087419,52.97294925,61.49857213,47.10543247,40.21301619,68.82726819,64.77979259,42.74833004,64.12742171,78.34564846,64.60056236,83.37291396,55.02420962,47.53439493]

x_dots = []

for i in range(1, 126):
  x_dots.append(i)

# Plot all lines
plt.plot(x_dots, y1_dots, label='Stdev 0.1')
plt.plot(x_dots, y2_dots, label='Stdev 0.5')
plt.plot(x_dots, y3_dots, label='Stdev 0.9')
# Add legend, x and y labels, and titles for the lines
plt.legend()
plt.xlabel('Trials')
plt.ylabel('Average Fitness')
plt.title('Average Fitness per Trial in Full Microbial Algorithm')
# Display the plot
plt.show()


In [ ]:
# Full microbial p-crossover

y1_dots = [52.18988974,35.58649066,47.89104532,42.93405591,56.5726234,52.52624959,36.47224373,42.25693961,40.65939932,73.35991839,55.03852892,61.35594263,35.33442847,53.82223469,35.48737456,73.15513929,53.76478237,53.28534921,33.34320125,43.98636907,74.03877519,37.2129392,37.87095023,36.60946949,43.34315889,36.0781091,44.92201991,51.85015031,49.8201582,70.29011357,48.33227403,62.97177525,42.37400951,33.98501414,40.89979332,49.67629664,56.67284944,38.74044371,79.91298112,38.2075163,43.40034082,64.83444617,30.94236423,42.09683677,40.7338673,38.75208276,42.29929453,55.06697564,65.5419297,42.51956017,44.23876379,79.21737089,73.80151717,36.20021142,40.99240763,73.50687353,40.75563371,44.67528916,31.13610424,71.93867453,64.87451318,54.71448991,37.01998784,35.06628254,47.21833196,48.67482626,54.7254617,51.34072546,64.98381631,38.28081804,54.18979471,47.74333886,44.74298188,40.5157388,57.67267977,44.69977892,61.60546976,42.98210464,65.65752513,62.02912508,44.31856766,54.82532974,37.6592334,36.62895339,38.14476663,34.82792353,34.79123319,43.28099399,54.55001403,50.54581592,75.91365233,55.20096199,45.05629632,44.67080619,44.72216161,36.28895624,40.34508403,58.03040006,51.0865242,65.6770353,46.62326357,38.85388517,71.56172135,54.53096692,55.08777854,43.5985772,63.45072955,54.90781548,41.37741577,37.32153622,63.10537407,56.6361249,65.03805615,37.60966736,65.17143563,47.13497752,39.60917801,44.03219113,33.273965,44.47580525,41.59767968,59.56427724,38.2328383,49.55421344,43.00783468]
y2_dots = [59.17475721,42.49860718,72.0601496,67.04744811,39.3035248,65.40080589,79.39649481,78.27504197,49.09417747,54.81905314,80.17862539,78.23768606,39.04695985,48.32516639,71.08306281,46.82967569,49.61719119,77.3650366,60.26939123,69.0570344,61.39436241,46.47322275,70.42563525,68.69148513,64.38894899,59.53296838,72.72649726,57.81855293,52.48368141,47.45260248,73.65689822,55.53062424,57.22462249,57.77106643,60.23818309,71.56813995,40.53044514,72.31297344,58.24669111,73.763001,76.29702762,55.93049889,55.78313603,62.78896305,81.73498629,82.9074156,55.41771524,75.85147905,60.10859553,65.9755061,71.93271776,75.46190162,41.93054703,62.17217734,66.79716819,56.68248485,80.13068022,62.75936332,62.38589965,61.8634567,56.83207025,74.00814183,45.15781821,40.47751912,44.16339339,37.65053329,44.94217378,59.99236624,51.35475701,44.28282916,44.48753062,64.91776507,60.07280406,46.42842232,38.55045626,62.66887217,62.91220731,72.61205673,63.83438809,73.82867046,64.99133937,64.31366129,50.7427109,58.11313536,48.16209133,51.95904733,64.54225949,51.66947835,54.30103475,56.2882707,38.99838021,68.91419404,62.75102542,65.91719421,53.07618293,82.10367098,79.98293058,43.59265167,84.22104774,40.13200433,50.76286665,66.73864101,49.77201542,73.37188733,42.23439967,61.14541113,57.00906685,50.82806455,60.38267529,48.07270457,59.9887079,39.54061648,64.42426157,51.72660872,61.09458408,50.50738354,78.63881072,50.75596713,56.47422676,66.56034954,61.00923876,74.3358028,65.60515133,67.85499041,57.89050592]
y3_dots = [68.8394883,55.99639566,71.60275964,42.76505508,62.15875343,58.73730433,74.87040903,50.83490546,37.28501744,55.00576694,50.42606508,59.48645688,47.34493749,64.31114883,37.22895072,60.11802572,81.83573873,56.89087741,39.71890081,44.00397411,41.27483722,62.02095869,57.33604632,42.39354788,55.18820664,57.59921681,54.1718628,64.4999931,57.39763151,44.05339941,47.62788064,41.54169326,65.40403061,41.50971054,52.41249937,60.31620141,55.32989091,71.241628,68.2203019,61.34213957,46.45605083,63.442297,51.74209172,61.40039175,62.33088053,37.0462201,40.5976777,55.34471105,56.66994915,75.3276339,39.28676195,66.62176887,42.19676533,66.66339349,64.74963811,45.42100429,57.86010373,48.16624287,58.8250681,46.81573036,42.43604129,59.85566339,80.77627128,64.72449147,55.88195819,60.75593091,71.16416486,52.27880542,39.79931382,66.30806167,52.68269671,53.97078608,59.39228366,62.86924401,81.16554118,48.10341911,76.45133311,69.2749814,81.10398763,52.44628506,51.57844657,43.47336128,37.92612362,64.97102754,58.30977801,65.90463501,40.94100245,68.15909904,50.99748635,41.04776404,45.40019488,44.65538878,71.38231525,59.78336362,83.90374806,66.27805498,68.12768781,52.90150248,67.34181939,55.76418285,49.68227498,61.64836202,66.65692418,49.08684909,53.97959823,50.15833531,36.79650207,79.86041479,48.4437663,59.67061548,64.74226085,45.18425194,53.65255501,45.08596259,58.39468488,51.50819345,65.22770033,74.08599738,62.46696502,56.83243175,50.56994409,66.8491189,66.56888906,76.74704253,37.61216496]

x_dots = []

for i in range(1, 126):
  x_dots.append(i)

# Plot all lines
plt.plot(x_dots, y1_dots, label='P-crossover 0.1')
plt.plot(x_dots, y2_dots, label='P-crossover 0.5')
plt.plot(x_dots, y3_dots, label='P-crossover 0.9')
# Add legend, x and y labels, and titles for the lines
plt.legend()
plt.xlabel('Trials')
plt.ylabel('Average Fitness')
plt.title('Average Fitness per Trial in Full Microbial Algorithm')
# Display the plot
plt.show()

In [ ]:
# P crossover

# Creating dataset
stats = ['0.5', '0.1', '0.9']

data = [50.4, 20.80, 28.80]

# Creating plot
myexplode = [0.05, 0, 0]
fig = plt.figure(figsize=(10, 7))

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/105.0))
        return '{p:.2f}%'.format(p=pct,v=val)
    return my_autopct

plt.title('Percent of Trials Won by Each Crossover Rate')

plt.pie(data, labels=stats, explode = myexplode, shadow = True, autopct=make_autopct(data))
plt.legend()

# Show plot
plt.show()

In [ ]:
# Mutation, Full microbial

# Creating dataset
stats = ['0.5', '0.1', '0.9']

data = [32, 30.4, 37.6]

# Creating plot
myexplode = [0.05, 0, 0]
fig = plt.figure(figsize=(10, 7))

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/105.0))
        return '{p:.2f}%'.format(p=pct,v=val)
    return my_autopct

plt.title('Percent of Trials Won by Each Stdev in Full Microbial')

plt.pie(data, labels=stats, shadow = True, autopct=make_autopct(data))
plt.legend()

# Show plot
plt.show()

In [ ]:
# Mutation, Hillclimber

# Creating dataset
stats = ['0.1', '0.5', '0.9']

data = [9.6, 49.6, 40.8]

# Creating plot
myexplode = [0.05, 0., 0.]
fig = plt.figure(figsize=(10, 7))

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/105.0))
        return '{p:.2f}%'.format(p=pct,v=val)
    return my_autopct

plt.title('Percent of Trials Won by Each Stdev in Hillclimber')

plt.pie(data, labels=stats, shadow = True, autopct=make_autopct(data))
plt.legend()

# Show plot
plt.show()

## Future experimentation, Hill Climber with 10 individuals

In [ ]:
################################################################################

# 1.3 Implement a hill climber that evolves the genotype
num_individuals = 10
generations = 1000

# set the starting position and the bearing
starting_position = (3,3)
starting_bearing = 10 #bearing in degrees
runtime = 5 #run to 5 seconds

genotypes = np.zeros((num_individuals, num_genes))

for i in range(num_individuals):
  genotypes[i] = generate_genotype()

# Number of generations
for g in range(generations):

  # Copy the genotype to edit
  copy_geno = np.copy(genotypes)

  # Set up all the agents
  agents = []
  for i in range(num_individuals):
    agent = Braitenberg(starting_position, starting_bearing, genotypes[i])
    agents.append(agent)

  env=environment() #create the environment that the agent will run in

  # Calculate original fitness
  og_fit = np.zeros(num_individuals)
  for x in range(num_individuals):
    trajectory,intensities = env.run(runtime, agents[x],show=False)
    og_fit[x] = fitness(trajectory, intensities, lightsource)

  # For however many individuals there are
  for i in range(num_individuals):
    # Mutate the genotype
    copy_geno[i] = mutate(genotypes[i])

  # Set up all the new agents
  agents = []
  for i in range(num_individuals):
    agent = Braitenberg(starting_position, starting_bearing, copy_geno[i])
    agents.append(agent)

  # Calculate the new fitness
  new_fit = np.zeros(num_individuals)
  for x in range(num_individuals):
    trajectory,intensities = env.run(runtime, agents[x],show=False)
    new_fit[x] = fitness(trajectory, intensities, lightsource)

  #print("OG: ", og_fit, " New: ", new_fit)

  # If the new fitness is better than the old, replace that line
  successful_individuals = 0
  for x in range(num_individuals):
    if new_fit[x] > og_fit[x]:
      genotypes[x] = copy_geno[x]
      successful_individuals += 1


  #print("Number of successful individuals on this run: ", successful_individuals)

# Find the best genotype
best_idx = 0
for x in range(num_individuals):
  if og_fit[x] > og_fit[best_idx]:
    best_idx = x

# Run and plot the best genotype
agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
trajectory,intensities = env.run(runtime, agent,show=True)
print("Fitness of the best individual: ", og_fit[best_idx])
print("Genotype of the best individual: ", genotypes[best_idx])

# Print the average fitness
iterations = 1000
total_fitness = 0

for i in range(iterations):
  agent = Braitenberg(starting_position, starting_bearing, genotypes[best_idx])
  trajectory,intensities = env.run(runtime, agent,show=False)
  total_fitness += fitness(trajectory, intensities, lightsource)

avg_fitness1 = total_fitness / iterations
print("\nAverage fitness: ", avg_fitness1)

# Physical robot


We need to download the robot control library and upload it to the board.

In [ ]:
!wget https://raw.githubusercontent.com/shepai/OpenEduBot/main/Library/EduBot.py

#Sim2Real

The following is the code that is needed on board, which will need to be copied across and saved as main.py

In [ ]:
"""
Braightenberg robot for crossing the reality gap
The light sensors allow light following behaviours.

The sensor pins are read at 26 and 27, if the robot is not turning away from the collision,
consider switching the pins around (26 & 27).

Code by Dexter R Shepherd

"""

from EduBot_CP import wheelBot
# from EduBot import wheelBot # Use this if your library is named EduBot instead of EduBot_CP
import time
import board
from analogio import AnalogIn

# IMPORTANT: Depending on your robot, you'll need to switch this board_type value between "pico", "pico_1" and "default"
# - "pico" board types will contain a small, long green raspbery pi circuit board.
# - "default" board types are a lot larger and purple coloured
bot = wheelBot(board_type="pico")

# Declare Sensor variables
s1 = AnalogIn(board.GP27)
s2 = AnalogIn(board.GP26)

# Change the sensor gain to preference
sensor_gain=0.5

def get_intensity(pin):
    return (pin.value*3.3) /65536

def calc(genotype):
    # Assign network weights & biases from genotype
    w_ll,w_lr,w_rl,w_rr,bl,br = (genotype[0],genotype[1],genotype[2],genotype[3],genotype[4],genotype[5])

    # Calculate local intensity
    il,ir= (34 - get_intensity(s1)*10, 34 - get_intensity(s2)*10)

    #Motor outputs = weights times inputs plus bias
    lm = il*w_ll + ir*w_rl + bl;
    rm = il*w_lr + ir*w_rr + br;

    # For debugging
    print("Sensors >",il,ir)
    print("Motors >",lm,rm)

    return lm,rm

# Insert your evolved genotype here
#geno = [0, 0, 0, 0, 1, 1]
# geno = [3.1256221,  4.99, 1.99002829, 2.74792338, 4.99, 4.29326977] # Hillclimber mutation 0.1 DONE
# geno = [2.59410866, 4.99, 4.99, 4.44998225, 4.99, 4.46335378] # Hillclimber mutation 0.9 DONE
# geno = [2.47671308, 4.58318036, 3.29549541, 4.99, 4.99, 4.1115206] # Full Microbial mutation 0.9
# geno = [1.27274977, 4.99, 4.95202981, 4.99, 4.99, 4.14478512] # Full Microbial pcrossover 0.1
# geno = [1.3793756, 4.98454823, 4.32232123, 4.04753596, 4.99, 4.18829005] # Full microbial pcrossover 0.5
geno = [0.78239848, 4.17036456, 4.99, 4.99, 4.99, 4.18284301] # Full Microbial pcrossover 0.9

# Change this to modify motor outputs to appropriate speeds
motorSensitivity = 0.25

#Bridging the reality gap, making sure it moves in a straight line
leftAdjustment = 0.12 * motorSensitivity

while True:
    # Calculate motor speeds for this frame depending on intensities
    speedLeft, speedRight = calc(geno)

    # Command the motors to spin - board_type = "pico"
    # NB: You may need to swap these parameters around depending on how your physical robot is configured
    # e.g. motor numbers could be anything from 1-4; if your robot is going backwards when it should be going forwards, change "f" to "r"
    bot.motorOn(4, "r", motorSensitivity * (speedLeft))
    bot.motorOn(3, "r", motorSensitivity * speedRight)

    # Command the motors to spin - board_type = "default" (uncomment if your board_type is "default")
    # NB: multiple speed by -1 to reverse direction. motor1 may be correspond to left or right motor depending on your robot's configuration.
    """
    bot.motor1_move(speedRight * motorSensitivity)
    bot.motor2_move(speedLeft * motorSensitivity)
    """

    # Wait 0.1 seconds to execute again
    time.sleep(0.1)
